# Programming HW1 Attempt

In [1]:
def read_baskets():
    """
    Reading input
    Method 1
    """
    baskets_list = []
    with open("browsingdata.txt", "r") as f:
        lines = f.readlines()
    for line in lines:
        baskets_list.append(line.split())
    return baskets_list

In [2]:
import csv

def read_baskets_II(fname):
    """
    Function to read from file
    csv library
    """
    record = []
    with open(fname) as input_file:
        lines = csv.reader(input_file, delimiter=' ')
        for row in lines:
            record.append(row[:-1])
    return record

In [3]:
def init_C1(baskets_list):
    item_dict = {}
    for basket in baskets_list:
        for item in basket:
            if item not in item_dict:
                item_dict[item] = 1
            else:
                item_dict[item] += 1
    return item_dict

In [4]:
def init_L1(item_dict, support=100):
    L1 = {}
    for key, value in item_dict.items():
        if value >= support:
            L1[key] = value
    return L1

In [5]:
import collections 

def init_L1_II(baskets_list, support=100):
    counter_dict = collections.Counter()
    for basket in baskets_list:
        for item in basket:
            counter_dict[item] += 1
            
    L1 = {i:counter_dict[i] for i in counter_dict if counter_dict[i] >= support} #Comprehension Syntax
    
    return L1

SyntaxError: invalid syntax (<ipython-input-5-f2bc8cf7147d>, line 9)

In [118]:
#Illustration of comprehensive lists

import numpy as np

def gaussian_neighbour(a):
    mu, sigma = 0, 1
    noise = np.random.normal(mu, sigma, 1)[0]
    return a + noise

gaus_neighbors = [gaussian_neighbour(5) for _ in range(20) ]
#gaus_neighbors
#gaussian_neighbour(5)

In [80]:

def generateLk(Lk_1, baskets_list, k, support=100):
    set_of_keys = []
    a=0
    
    for key, _ in Lk_1.items():
        if type(key)==tuple:
            for elem in key:
                if elem not in set_of_keys:
                    set_of_keys.append(elem)
        else:
            if key not in set_of_keys:
                set_of_keys.append(key)
                
    Lk = {}
    for item in itertools.combinations(set_of_keys, k):
        Lk[tuple(sorted(item))] = 0
    for basket in baskets_list:
        basket = sorted(basket)
        combs_iterator = itertools.combinations(basket, k)
        for combs in combs_iterator:
            if combs in Lk.keys():
                Lk[combs]+=1
    Lk = {i:Lk[i] for i in Lk if Lk[i] >= support}
    return Lk

In [127]:
#sorted(a, key, reverse)
def second_char(a):
    return a[1]

a =['av', 'ba', 'cd', 'pr']
sorted(a, key=lambda x:x[1])
#sorted(a, reverse=True)

['ba', 'cd', 'pr', 'av']

In [81]:
def generateLk_II(baskets_list, k, support=100):
    counter_dict = collections.Counter()
    for basket in baskets_list:
        basket = sorted(basket)
        combs_iterator = itertools.combinations(basket, k)
        for combs in combs_iterator:
            counter_dict[combs]+=1
    Lk = {i:counter_dict[i] for i in counter_dict if counter_dict[i] >= support}
    return Lk

In [82]:
def top_k_sorted_dict(dict_, k):
    s_dict = list(sorted(dict_.items(), key=lambda item: item[1]))
    return s_dict[(-1-k):-1]

In [83]:
def confidenceScore_k(Lk, Lk_1, k, k_top=5):
    temp = {}
    conf_score = collections.Counter()
    if k>2:
        for x in Lk_1:
            Lk_1[tuple(sorted(x))] = Lk_1[x]
    for x in Lk:
        x=tuple(sorted(x))
        tuple_rotator = collections.deque(x)
        for _ in range(k):
            tpl = tuple(tuple_rotator)
            tpl = tuple(sorted(tpl[:-1]))+(tpl[-1],)
            temp[tpl] = Lk[x]
            tuple_rotator.rotate()
    for elem in temp:
        if k==2:
            conf_score[elem] = temp[elem] / float(Lk_1[elem[0]])
        else:
            conf_score[elem] = temp[elem] / float(Lk_1[elem[:-1]])
    return top_k_sorted_dict(conf_score, k_top)


### Main Program

In [84]:
import time
import collections #pecialized container subclass from dict
import itertools #iterators for efficient looping


baskets = read_baskets()
t1 = time.time()
C1 = init_C1(baskets)
L1 = init_L1(C1)
L2 = generateLk(L1, baskets, 2)
#L22 = generateLk_II(baskets, 2)
L3 = generateLk(L2, baskets, 3)
#L33 = generateLk_II(baskets, 3)
cs = confidenceScore_k(L3, L2, 3)
t2 = time.time()
print("Exec time: {:.2f} min.".format((t2-t1)/60))

Exec time: 0.10 min.


In [137]:
cs=[0,1,2,3]
#cs.reverse()
cs[::1]

[0, 1, 2, 3]

##### Timely-worse function example

In [67]:
def bad_generateL3(basket_list, L2, support=100):
    kcombination = []
    Lk = {}
    for a in L2:
        for b in L2:
            if(a!=b):
                if set(a) & set(b) and len(list(set(a) & set(b))) >= 1:
                    kcombination.append(sorted(set(a)|set(b)))
    for k in range(0, len(kcombination)):
        for basket in basket_list:
            if kcombination[k][0] in basket:
                if kcombination[k][1] in basket:
                    if kcombination[k][2] in basket:
                        if tuple(kcombination[k]) not in Lk.keys():
                            Lk[tuple(kcombination[k])] = 1
                        else:
                            Lk[tuple(kcombination[k])] += 1
    return Lk

In [69]:
t1 = time.time()
L3 = generateLk(L2, baskets, 3)
t2 = time.time()
print("Exec time: {:.2f} min.".format((t2-t1)/60))
t1 = time.time()
L3_bad = bad_generateL3(baskets, L2)
t2 = time.time()
print("Exec time: {:.2f} min.".format((t2-t1)/60))

Exec time: 0.08 min.
Exec time: 15.40 min.


### General Observations

- Avoid unnecessary comparisons when possible 
<br>
- Avoid conversion back and forth between string type and other types
<br>
- Avoid using strings whenever possible
<br>
- Translation is always good: 
>'FRO11987 ' --> 1 <br>
> . . . <br>
> 'ELE59935 ' --> 45

In [138]:
def encode(inFile, outFile):
    encoding_dict = {}
    enc_index = 0
    with open(inFile) as infile:
        with open(outFile, 'w') as outfile:
            for line in infile:
                new_fields = []
                fields = line.split(' ')
                for element in fields:
                    if element=='\n':
                        continue
                    elif element not in encoding_dict:
                        encoding_dict[element] = str(enc_index)
                        enc_index += 1
                        new_fields.append(encoding_dict[element])
                    else:
                        new_fields.append(encoding_dict[element])
                outfile.write(','.join(new_fields))
                outfile.write('\n')
    return encoding_dict
encoding_dict = encode('browsingdata.txt', 'encoded.csv')

- Use Python packages to automate actions, they are pre-optimized.
<br>
- When using lists of lists, Numpy.array are the way to go.

#### Pandas library 
Pandas is a very good library for handling large representation of complex data collections

- Calculate statistics and answer questions about the data, like average, inter-columns correlations, distributions of items...
- Good handle on tabular data with heterogeneously-typed columns.
- Visualize the data with help from Matplotlib. 
- Efficient storage and reading data (CSV, json, databases...)

In [145]:
import pandas as pd

dFrame = pd.read_csv("movies.csv")
d=dFrame[:10]
d

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [98]:
genres_set = set()
movies = set()
for i in d.index:
    line = str(d.at[i, 'genres']) #grab item at row i and column 'genres'
    genres_set.update(line.split('|'))
    line = str(d.at[i, 'title'])
    movies.update(line.split('|'))
    
genres_dFrame = pd.DataFrame(index=genres_set, columns=movies)
genres_dFrame

,Toy Story (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),Jumanji (1995),Grumpier Old Men (1995),GoldenEye (1995),Heat (1995)
Romance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Drama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Thriller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Children,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adventure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Crime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Animation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
import numpy as np

matrix = np.random.randint(0, 6, genres_dFrame.shape)

In [102]:
genres_dFrame = pd.DataFrame(matrix, index=genres_set, columns=movies)
genres_dFrame

,Toy Story (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),Jumanji (1995),Grumpier Old Men (1995),GoldenEye (1995),Heat (1995)
Romance,4,4,5,5,2,5,3,2,0,1
Comedy,3,5,4,4,0,4,5,4,3,5
Drama,2,1,3,1,5,0,4,4,4,4
Thriller,1,3,2,5,4,3,3,1,5,1
Fantasy,4,0,3,0,0,0,2,5,0,3
Action,3,5,4,5,3,3,5,1,2,3
Children,3,5,2,4,3,4,4,2,2,0
Adventure,5,1,3,4,1,0,2,5,1,0
Crime,4,5,1,5,3,2,0,3,4,0
Animation,3,2,5,2,5,4,2,5,5,1
